In [2]:
import pandas as pd
import numpy as np
import os
from ast import literal_eval
import torch_geometric
from torch_geometric.data import Data
import networkx as nx
import torch
from transformers import GraphormerModel, GraphormerConfig
from transformers import T5EncoderModel
from transformers import T5Tokenizer
SEED = 42
import random
torch.manual_seed(SEED)
torch.random.manual_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.cuda.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

ModuleNotFoundError: No module named 'torch_geometric'

In [3]:
# pip install torch_geometric datasets transformers accelerate

In [3]:
from torch.utils.data import Dataset

class QuestionAnswerDataset(Dataset):

    def __init__(self, df, tokenizer, max_length, context_key="answerEntity",
                 tokenizer_truncation="only_first"):
        super(QuestionAnswerDataset).__init__()

        self.questions = df.question.values
        self.contexts = df[context_key].values
        self.labels = torch.tensor(df.correct.values, dtype=torch.float32)#torch.tensor([0]*10961, dtype=torch.float32)##torch.tensor(df.correct.values, dtype=torch.float32)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.tokenized_input = [tokenizer.encode_plus(x, y,
                              max_length=self.max_length,
                              padding="max_length",
                              truncation=tokenizer_truncation,
                              return_tensors="pt", ) \
                          for x, y in zip(self.questions,
                                          self.contexts)]
        print(len(self.questions))
        print(len(self.contexts))
        print(len(self.labels))
        assert len(self.questions) == len(self.contexts) == len(self.labels)

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return {
                "input_ids": self.tokenized_input[idx]["input_ids"][0],
                "attention_mask" : self.tokenized_input[idx]["attention_mask"][0],
                "labels": self.labels[idx]}

In [4]:
training_data = "/kaggle/input/kbqa-classification/train.tsv"#"/kaggle/input/kbqa-classification/train.tsv"#"/kaggle/input/kbqa-classification/train.tsv"#"/kaggle/input/kbqa-classification/train.tsv"
train_data = pd.read_csv(training_data, sep="\t")
train_data["graph"] = train_data["graph"].apply(eval)
train_data.head(2)

,sample_id,question,questionEntity,answerEntity,groundTruthAnswerEntity,answerEntityId,questionEntityId,groundTruthAnswerEntityId,correct,graph
0,0,Whst is the name of the head of state and high...,Iran,Ruhollah Khomeini's return to Iran,Office of the Supreme Leader of Iran,Q7293530,Q794,Q16045000,False,"{'nodes': [{'type': 'QUESTIONS_ENTITY', 'name_..."
1,1,Whst is the name of the head of state and high...,Iran,Ruhollah Khomeini's letter to Mikhail Gorbachev,Office of the Supreme Leader of Iran,Q5952984,Q794,Q16045000,False,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q417..."


In [7]:
# train_data['correct']=train_data['GT']

In [8]:
# graph_json = train_data.iloc[0].graph
# graph_json['directed']=True
# nx_graph = nx.node_link_graph(graph_json, )


In [9]:
# nx_graph

In [10]:
def linearize_graph(graph_dict):
    nodes = sorted((node_dict for node_dict in graph_dict["nodes"]), key=lambda d:d["id"])
    for n_id, node_dict in enumerate(nodes):
        assert n_id == node_dict["id"]
    src_node_id2links = {}
    # print("graph_dict", graph_dict)
    # print("links", graph_dict["links"])
    for link_dict in graph_dict["links"]:
        link_src =  link_dict["source"]
        if src_node_id2links.get(link_src) is None:
            src_node_id2links[link_src] = []
        src_node_id2links[link_src].append(link_dict)
    graph_s = ""
    # print("src_node_id2links", src_node_id2links)
    for n_id, node_dict in enumerate(nodes):
        links = src_node_id2links.get(n_id, list())
        start_label = node_dict["label"]
        if node_dict["type"] == "ANSWER_CANDIDATE_ENTITY":
            start_label = f"{SEP_TOKEN} {start_label} {SEP_TOKEN}"
        for link_dict in links:
            target_label = nodes[link_dict["target"]]["label"]
            if nodes[link_dict["target"]]["type"] == "ANSWER_CANDIDATE_ENTITY":
                target_label = f"{SEP_TOKEN} {target_label} {SEP_TOKEN}"
            link_s = f" {start_label}, {link_dict['label']}, {target_label} "
            graph_s += link_s
        # graph_s += node_dict["label"]
        # print("n_id, node_dict", n_id, node_dict)
        # if n_id != len(nodes) - 1:


        #     link_label = link["label"]
        #     graph_s += link_label
    # print('--')
    return graph_s

In [11]:
# train_data['qa'] = train_data['question']+" ; "+train_data['answerEntity']
tokenizer = T5Tokenizer.from_pretrained("DeepPavlov/t5-wikidata5M-with-neighbors")#("s-nlp/t5_large_ssm_nq_mintaka")#("DeepPavlov/t5-wikidata5M-with-neighbors")
# qa_tokens = []
SEP_TOKEN = tokenizer.sep_token
train_data["linearized_graph"] = train_data["graph"].apply(linearize_graph)


tokenizer_config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [1]:
train_data[27]

NameError: name 'train_data' is not defined

In [12]:
train_data['linearized_graph'][27]

' English, on focus list of Wikimedia project, Wikipedia:Vital articles/Level/4  World War I, on focus list of Wikimedia project, Wikipedia:Vital articles/Level/4  [SEP] Franz Ferdinand [SEP], language of work or name, English '

In [13]:
train_data['graph'][27]

{'nodes': [{'type': 'INTERNAL',
   'name_': 'Q6173448',
   'id': 0,
   'label': 'Wikipedia:Vital articles/Level/4'},
  {'type': 'INTERNAL', 'name_': 'Q1860', 'id': 1, 'label': 'English'},
  {'type': 'QUESTIONS_ENTITY',
   'name_': 'Q361',
   'id': 2,
   'label': 'World War I'},
  {'type': 'ANSWER_CANDIDATE_ENTITY',
   'name_': 'Q829973',
   'id': 3,
   'label': 'Franz Ferdinand'}],
 'links': [{'name_': 'P5008',
   'source': 1,
   'target': 0,
   'label': 'on focus list of Wikimedia project'},
  {'name_': 'P5008',
   'source': 2,
   'target': 0,
   'label': 'on focus list of Wikimedia project'},
  {'name_': 'P407',
   'source': 3,
   'target': 1,
   'label': 'language of work or name'}]}

In [296]:
# model_name="sentence-transformers/all-mpnet-base-v2"
# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# bert_model = AutoModel.from_pretrained(model_name)
# SEP_TOKEN = tokenizer.sep_token
# train_data["linearized_graph"] = train_data["graph"].apply(linearize_graph)


In [297]:
# train_data['correct'] = [0]*10961

In [305]:
train_dataset = QuestionAnswerDataset(train_data, tokenizer=tokenizer, max_length=128,context_key="linearized_graph",
                                      tokenizer_truncation="only_second")
train_dataset

37672
37672
37672


In [306]:
train_data['qa_tokens']=[dict(item)['input_ids'] for item in train_dataset]

In [139]:
# true_labels_df = train_data[train_data['correct'] ==True].copy()
# duplicated_true_labels = pd.concat([true_labels_df] * 3, ignore_index=True)

# # Concatenate the original DataFrame and duplicated true labels
# train_data = pd.concat([train_data, duplicated_true_labels], ignore_index=True)
# train_data.info()

In [140]:
# train_data.to_csv('prep_dataset.csv', index=False)

In [307]:
# # Copyright (c) Microsoft Corporation and HuggingFace
# # Licensed under the MIT License.

from typing import Any, Dict, List, Mapping


class GraphormerDataCollator:
    def __init__(self, spatial_pos_max=20, on_the_fly_processing=False):
        # if not is_cython_available():
            # raise ImportError("Graphormer preprocessing needs Cython (pyximport)")

        self.spatial_pos_max = spatial_pos_max
        self.on_the_fly_processing = on_the_fly_processing

    def __call__(self, features: List[dict]) -> Dict[str, Any]:
        # if self.on_the_fly_processing:
        #     features = [preprocess_item(i) for i in features]
        try:
            if not isinstance(features[0], Mapping):
                features = [vars(f) for f in features]
            batch = {}
            max_node_num = max(len(i["input_nodes"]) for i in features)
            node_feat_size = len(features[0]["input_nodes"][0])
            edge_feat_size = len(features[0]["attn_edge_type"][0][0])
            max_dist = max(len(i["input_edges"][0][0]) for i in features)
            edge_input_size = len(features[0]["input_edges"][0][0][0])
            batch_size = len(features)

            batch["attn_bias"] = torch.zeros(batch_size, max_node_num + 1, max_node_num + 1, dtype=torch.float)
            batch["attn_edge_type"] = torch.zeros(batch_size, max_node_num, max_node_num, edge_feat_size, dtype=torch.long)
            batch["spatial_pos"] = torch.zeros(batch_size, max_node_num, max_node_num, dtype=torch.long)
            batch["in_degree"] = torch.zeros(batch_size, max_node_num, dtype=torch.long)
            batch["input_nodes"] = torch.zeros(batch_size, max_node_num, node_feat_size, dtype=torch.long)
            batch["input_edges"] = torch.zeros(
                batch_size, max_node_num, max_node_num, max_dist, edge_input_size, dtype=torch.long
            )
            batch["qa_tokens"] = torch.zeros(batch_size,1, 128, dtype=torch.int)  # Initialize qa_tokens tensor

            for ix, f in enumerate(features):
                for k in ["attn_bias", "attn_edge_type", "spatial_pos", "in_degree", "input_nodes", "input_edges", "qa_tokens"]:
                    f[k] = torch.tensor(f[k])

                if len(f["attn_bias"][1:, 1:][f["spatial_pos"] >= self.spatial_pos_max]) > 0:
                    f["attn_bias"][1:, 1:][f["spatial_pos"] >= self.spatial_pos_max] = float("-inf")

                batch["attn_bias"][ix, : f["attn_bias"].shape[0], : f["attn_bias"].shape[1]] = f["attn_bias"]
                batch["attn_edge_type"][ix, : f["attn_edge_type"].shape[0], : f["attn_edge_type"].shape[1], :] = f[
                    "attn_edge_type"
                ]
                batch["spatial_pos"][ix, : f["spatial_pos"].shape[0], : f["spatial_pos"].shape[1]] = f["spatial_pos"]
                batch["in_degree"][ix, : f["in_degree"].shape[0]] = f["in_degree"]
                batch["input_nodes"][ix, : f["input_nodes"].shape[0], :] = f["input_nodes"]
                batch['qa_tokens'][ix] = f['qa_tokens'][0]
                batch["input_edges"][
                    ix, : f["input_edges"].shape[0], : f["input_edges"].shape[1], : f["input_edges"].shape[2], :
                ] = f["input_edges"]

            batch["out_degree"] = batch["in_degree"]
    #         batch["qa"]
            sample = features[0]["labels"]
            if len(sample) == 1:  # one task
                if isinstance(sample[0], float):  # regression
                    batch["labels"] = torch.from_numpy(np.concatenate([i["labels"] for i in features]))
                else:  # binary classification
                    batch["labels"] = torch.from_numpy(np.concatenate([i["labels"] for i in features]))
            else:  # multi task classification, left to float to keep the NaNs
                batch["labels"] = torch.from_numpy(np.stack([i["labels"] for i in features], axis=0))

            return batch
        except:
            pass

In [308]:
# import MultiModalDiscussionTransformer.mDT.src.data.pyg_datasets.pre_processing as p
from transformers.models.graphormer.collating_graphormer import preprocess_item #GraphormerDataCollator
# from MultiModalDiscussionTransformer.mDT.src.data import algos # import floyd_warshall, gen_edge_input  # Import Cython functions
import numpy as np
import json

def json_graph_to_pyg_data(G, graph_json):
    # Create a NetworkX graph from the JSON data
    # G = nx.Graph()

    # # Add nodes with features
    # for node in json_graph['nodes']:
    #     G.add_node(node['id'], type=node['type'], name=node['name_'], label=node['label'])

    # # Add edges with attributes
    # for link in json_graph['links']:
    #     G.add_edge(link['source'], link['target'], name=link['name_'], label=link['label'])
    # G = nx.node_link_graph(graph_json, )
    # Convert NetworkX graph to PyTorch Geometric data
    edges = torch.tensor(list(G.edges()), dtype=torch.long).t().contiguous()
    # print(edges)
    edge_label_map = {label: idx for idx, label in enumerate(set(link['label'] for link in graph_json['links']))}
    # print(edge_label_map)
    edge_attrs = torch.tensor([edge_label_map[G[u][v][0]['label']] for u, v in list(G.edges())], dtype=torch.long).unsqueeze(1)

    # print(edge_attrs)
    x = torch.tensor([[node['id']] for node in graph_json['nodes']], dtype=torch.float)  # Using node IDs as features

    dist_matrix = np.zeros((len(G.nodes), len(G.nodes)), dtype=np.float32)
    for u, v, d in G.edges(data=True):
        dist_matrix[u][v] = 1  # For non-weighted graph, set distance as 1

    for i in range(len(G.nodes)):
        dist_matrix[i][i] = 0  # Diagonal elements

    dist_matrix = nx.floyd_warshall_numpy(G, weight=None)
    data = Data(node_feat=x, edge_index=edges, edge_attr=edge_attrs, distance_matrix=[dist_matrix], num_nodes = x.size(0), y=[0])

    return data

In [315]:
# Mine
from tqdm import tqdm
dataset = []
indexes = []
for ind, item in tqdm(train_data.iterrows()):
    try:
        graph_json = item.graph
        graph_json["directed"] = True
        G = nx.node_link_graph(graph_json, )

        data = json_graph_to_pyg_data(G, graph_json)
        data=preprocess_item(data)
#         print(data)
#         print(item.qa_tokens)
        data['qa_tokens'] = item.qa_tokens
        data.y =[1 if item.correct==True else 0]
        # data  = preprocess_item(data)
        for key in data.keys():
            data[key] = torch.tensor(data[key])
            data[key] = data[key].unsqueeze(0)
        dataset.append(data)
    except:
        print(ind)
        indexes.append(ind)
#         dataset.append(Data(x=[0]))
        pass

0it [00:00, ?it/s]/tmp/ipykernel_34/1209831208.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data[key] = torch.tensor(data[key])
319it [00:00, 649.92it/s]

190


2152it [00:03, 689.20it/s]

2072


2439it [00:03, 705.10it/s]

...

36249it [00:58, 640.79it/s]

36128


36778it [00:59, 612.94it/s]

36664


37672it [01:01, 615.49it/s]


In [316]:
from torch_geometric.loader import DataLoader
data_collator=GraphormerDataCollator()
loader = DataLoader(dataset, batch_size=1, collate_fn=data_collator, shuffle=False)

In [317]:
len(loader)

37541

In [318]:
generator1 = torch.Generator().manual_seed(SEED)
test_dataset, dev_dataset = torch.utils.data.random_split(dataset, [3700, 33841], generator=generator1)

In [319]:
train_loader = DataLoader(dev_dataset, batch_size=1, collate_fn=data_collator, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, collate_fn=data_collator, shuffle=False)

In [24]:
from datasets import DatasetDict
dataset_dict = DatasetDict({'train': dataset}, )

In [320]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttentionPooling(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(AttentionPooling, self).__init__()
        self.linear = nn.Linear(input_size, hidden_size)
        self.attention_weights = nn.Linear(hidden_size, 1)

    def forward(self, last_hidden_state):
        # Apply linear transformation to obtain attention scores
        attention_scores = self.attention_weights(torch.tanh(self.linear(last_hidden_state)))

        # Apply softmax to get attention weights
        attention_weights = F.softmax(attention_scores, dim=0)

        # Apply attention weights to node embeddings and sum to get graph embedding
        graph_embedding = torch.sum(torch.mul(last_hidden_state, attention_weights), dim=0)

        return graph_embedding


class InterHead(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.text_fc = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.graph_fc = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.softmax = nn.Softmax(dim=0)
        self.atn_layer = AttentionPooling(input_size=512, hidden_size=512)

    def forward(self, text, graph):
        # text = torch.mean(text[0], dim=0)
        # graph = self.atn_layer(graph[0])
        text_sen = self.text_fc(text)
        graph_sen = self.graph_fc(graph)

        a_tt = torch.mul(text, text_sen).sum(-1)
        a_tg = torch.mul(text, graph_sen).sum(-1)
        a_gt = torch.mul(graph, text_sen).sum(-1)
        a_gg = torch.mul(graph, graph_sen).sum(-1)

        a_tt, a_tg = self.softmax(torch.stack([a_tt, a_tg])).split([1, 1], dim=0)
        a_gt, a_gg = self.softmax(torch.stack([a_gt, a_gg])).split([1, 1], dim=0)

        a_tt = a_tt.squeeze(0).unsqueeze(-1)
        a_tg = a_tg.squeeze(0).unsqueeze(-1)
        a_gt = a_gt.squeeze(0).unsqueeze(-1)
        a_gg = a_gg.squeeze(0).unsqueeze(-1)

        text = torch.mul(a_tt, text) + torch.mul(a_tg, graph)
        graph = torch.mul(a_gt, text) + torch.mul(a_gg, graph)
        return text, graph

class TextModule(nn.Module):
    def __init__(self):
        super().__init__()
        T5EncoderModel._keys_to_ignore_on_load_unexpected = ["decoder.*"]
        self.auto_model = T5EncoderModel.from_pretrained("DeepPavlov/t5-wikidata5M-with-neighbors")
        self.tokenizer = T5Tokenizer.from_pretrained("DeepPavlov/t5-wikidata5M-with-neighbors")
        # self.auto_model = bert_model
    def forward(self, inputs):
        hidden_state = self.auto_model(inputs)
        return hidden_state['last_hidden_state']


class GraphormerModule(nn.Module):
    def __init__(self):
        super().__init__()
        config = GraphormerConfig(embedding_dim=512)
        self.model = GraphormerModel.from_pretrained("clefourrier/pcqm4mv2_graphormer_base", ignore_mismatched_sizes=True,config=config )
#         for layer in self.model.children():
#             if hasattr(layer, 'reset_parameters'):
#                 layer.reset_parameters()
    def forward(self, input_nodes, input_edges, attn_bias, in_degree, out_degree, spatial_pos, attn_edge_type):
        graph_encoding = self.model(input_nodes, input_edges, attn_bias, in_degree, out_degree, spatial_pos, attn_edge_type)
        outputs, hidden_states = graph_encoding["last_hidden_state"], graph_encoding["hidden_states"]
        return outputs, hidden_states

class T5GrapormerInteractModel(nn.Module):
    def __init__(self, layers=10, dropout=0.1):
        super().__init__()
        # self.layers=layers
        # self.dropout=droput
        self.text_encoder = TextModule()
        self.graph_encoder = GraphormerModule()
        for param in self.text_encoder.parameters():
            param.requires_grad = True
        for param in self.graph_encoder.parameters():
            param.requires_grad = True
        self.inter = nn.ModuleList()
        for i in range(10):
            self.inter.append(InterHead(hidden_dim=512))
        self.cls = nn.Linear(512, 1)
        self.act_func = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        # self.dense = nn.linear(512, 512)
        self.fc = nn.Linear(2 * 512, 512)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.Dropout(p=0.1),
            nn.ReLU(),
            nn.Linear(512, 1),
        )
        self.atn_layer=AttentionPooling(input_size=512, hidden_size=512)
    def forward(self, inputs):
        text_rep = self.text_encoder(inputs['qa_tokens'])
        # print(text_rep)
        graph_rep, _ = self.graph_encoder(torch.tensor(inputs['input_nodes']), torch.tensor(inputs['input_edges']), torch.tensor(inputs['attn_bias']), torch.tensor(inputs['in_degree']), torch.tensor(inputs['out_degree']), torch.tensor(inputs['spatial_pos']), torch.tensor(inputs['attn_edge_type']))
        # print(graph_rep)
#         print(text_rep, graph_rep)
        text_rep = torch.stack([elem[0, :] for elem in text_rep])#torch.stack([elem[0, :] for elem in text_rep])
        graph_rep = torch.stack([elem[0, :] for elem in graph_rep])#torch.stack([elem[0, :] for elem in graph_rep])
        org_text_rep = text_rep
        org_graph_rep = graph_rep
        for i in range(10-1):
            text_rep, graph_rep = self.inter[i](text_rep, graph_rep)
#         print(text_rep, graph_rep)
        text_code = org_text_rep+text_rep#torch.mul(org_text_rep, text_rep)
        graph_code = org_text_rep+graph_rep#torch.mul(org_graph_rep,graph_rep)
        text_code = self.dropout(self.act_func(text_code))
        graph_code = self.dropout(self.act_func(graph_code))
        rep = torch.cat([text_code,
                             graph_code],dim=-1)
#         print(rep)
        rep = self.act_func(self.dropout(self.fc(rep)))
        class_logits = self.classifier(rep)
#         class_logits = torch.sigmoid(class_logits)
        return class_logits






In [324]:
interact_model = T5GrapormerInteractModel()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of GraphormerModel were not initialized from the model checkpoint at clefourrier/pcqm4mv2_graphormer_base and are newly initialized: ['graph_encoder.graph_attn_bias.edge_dis_encoder.weight', 'graph_encoder.graph_attn_bias.edge_encoder.weight', 'graph_encoder.graph_attn_bias.graph_token_virtual_distance.weight', 'graph_encoder.graph_attn_bias.spatial_pos_encoder.weight', 'graph_encoder.graph_node_feature.atom_encoder.weight', 'graph_encoder.graph_node_feature.graph_token.weight', 'graph_encoder.graph_node_feature.in_degree_encoder.weight', 'graph_encoder.graph_node_feature.out_degree_encoder.weight', 'graph_encoder.layers.0.fc1.bias', 'graph_encoder.layers.0.fc1.weight', 'graph_encoder.layers.0.fc2.bias', 'graph_encoder.layers.0.fc2.weight', 'graph_encoder.layers.0.final_layer_norm.bias', 'graph_encoder.layers.0.final_layer_norm.weight', 'graph_encoder.layer

In [43]:
# interact_model.load_state_dict(torch.load('/content/drive/MyDrive/results_epoch16.pt', map_location=torch.device('cpu')))

In [28]:
# pip install wandb

In [322]:
import wandb
wandb.init(
    # set the wandb project where this run will be logged
    project="kadduformer",
)

avg_test_loss,▇█▁▃
avg_train_loss,█▄▂▁
epoch,▁▁▃▃▆▆██
training_loss,█▆▄▂▅▄▅▄▄▂▂▃▄▄▅▆▃▃▃▄▃▃▃▅▂▃▁▁▂▃▁▃▄▃▂▁▄▃▂▄
avg_test_loss,0.17928
avg_train_loss,0.16118
epoch,3
training_loss,0.18538


In [46]:
# from google.colab import drive
# drive.mount('/content/drive')

In [323]:
import numpy as np

def calculate_pos_weights(num_positives, num_negatives):
  """
  Calculates class weight for positive class in binary crossentropy with logits loss.

  Args:
      num_positives: int Number of positive samples.
      num_negatives: int Number of negative samples.

  Returns:
      float Class weight for the positive class.
  """
  if num_positives == 0:
    raise ValueError("Number of positive samples cannot be zero.")
  pos_weight = num_negatives / num_positives
  return pos_weight

# Example usage
num_positives = 3754
num_negatives = 33918
pos_weight = calculate_pos_weights(num_positives, num_negatives)
print(pos_weight)

9.035162493340437


In [ ]:
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
# weight = torch.tensor()
optimizer = optim.Adam(interact_model.parameters(), lr=3e-5,weight_decay=1e-5)

# Move model to device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCELoss()#BCEWithLogitsLoss()#(pos_weight = torch.tensor([.9035162493340437]).to(device))#(weight=torch.tensor([0.09964960713527288]))#(pos_weight = torch.tensor([0.09964960713527288]))  # Binary Cross-Entropy Loss

interact_model.to(device)
train_dataloader=train_loader
# Training loop
print_frequency = 1000
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce LR by a factor of 0.1 every 5 epochs
epochs = 4

for epoch in range(epochs):
    interact_model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    step_counter = 0
    running_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
#         if step_counter<20:
            try:

                optimizer.zero_grad()
                inputs, labels = batch.to(device),torch.tensor(batch['y'], dtype=torch.float).squeeze(1).to(device)

                # Forward pass
                outputs = interact_model(inputs)
#                 print(outputs[0])
#                 print(labels) # Remove the second dimension
                loss = criterion(outputs[0], labels)
                total_loss += loss.item()
                running_loss +=loss.item()
        #             grads = torch.autograd.grad(outputs=outputs, inputs=inputs, grad_outputs=torch.ones_like(outputs), allow_unused=True)

                loss.backward()
                  # Monitor gradients
                optimizer.step()

                step_counter += 1

                if step_counter % print_frequency == 0:
                    epoch_loss = total_loss / print_frequency  # Calculate average loss for the last print_frequency steps
                    print(f"output:{outputs.float().to(device)} , act: {labels.float()}")  # Calculate average loss for the last print_frequency steps
                    print("loss::::::::::::::::::::",epoch_loss)
                    wandb.log({"training_loss": epoch_loss})

                    total_loss = 0  # Reset total loss after printing
            except:
#                 print('error')
                pass

    scheduler.step()
# Compute average training loss for the epoch
    avg_train_loss = running_loss / len(train_loader)

    # Log average training loss for the epoch
    wandb.log({"epoch": epoch, "avg_train_loss": avg_train_loss})

    # Set model to evaluation mode
    interact_model.eval()

    # Evaluate on test data loader
    test_loss = 0.0
    with torch.no_grad():
        for test_batch in test_loader:
            try:
                inputs, labels = test_batch.to(device),torch.tensor(test_batch['y'], dtype=torch.float).squeeze(0).to(device)
                output = interact_model(inputs)
                test_loss += criterion(output[0], labels).item()
            except:
                pass
    avg_test_loss = test_loss / len(test_loader)

    # Log test loss for the epoch
    wandb.log({"epoch": epoch, "avg_test_loss": avg_test_loss})

    # Set model back to training mode
    interact_model.train()

    # Print epoch results
    torch.save(interact_model.state_dict(), f"results_epoch{epoch}.pt")
    print(f'Epoch {epoch + 1}, Avg. Training Loss: {avg_train_loss}, Avg. Test Loss: {avg_test_loss}')


Epoch 1/4:   0%|          | 0/33841 [00:00<?, ?it/s]/tmp/ipykernel_34/3446899099.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs, labels = batch.to(device),torch.tensor(batch['y'], dtype=torch.float).squeeze(1).to(device)
/tmp/ipykernel_34/4018292533.py:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  graph_rep, _ = self.graph_encoder(torch.tensor(inputs['input_nodes']), torch.tensor(inputs['input_edges']), torch.tensor(inputs['attn_bias']), torch.tensor(inputs['in_degree']), torch.tensor(inputs['out_degree']), torch.tensor(inputs['spatial_pos']), torch.tensor(inputs['attn_edge_type']))
Epoch 1/4:   3%|▎         | 1002/33841 [01:17<46:29, 11.77it/s]

output:tensor([[0.0306]], device='cuda:0', grad_fn=<SigmoidBackward0>) , act: tensor([0.], device='cuda:0')
loss:::::::::::::::::::: 0.28899315743305487


Epoch 1/4:   6%|▌         | 2002/33841 [02:37<42:59, 12.34it/s]

output:tensor([[0.0003]], device='cuda:0', grad_fn=<SigmoidBackward0>) , act: tensor([0.], device='cuda:0')
loss:::::::::::::::::::: 0.23086554735901882


Epoch 1/4:   9%|▉         | 3001/33841 [04:01<42:33, 12.08it/s]

...

Epoch 3/4:  68%|██████▊   | 23004/33841 [30:03<14:29, 12.47it/s]

output:tensor([[0.9542]], device='cuda:0', grad_fn=<SigmoidBackward0>) , act: tensor([1.], device='cuda:0')
loss:::::::::::::::::::: 0.17730856279509224


Epoch 3/4:  71%|███████   | 24004/33841 [31:22<12:46, 12.84it/s]

output:tensor([[3.3140e-05]], device='cuda:0', grad_fn=<SigmoidBackward0>) , act: tensor([0.], device='cuda:0')
loss:::::::::::::::::::: 0.19389816702651352


Epoch 3/4:  73%|███████▎  | 24650/33841 [32:12<11:44, 13.05it/s]